In [1]:
#imports
from tensorflow.keras.datasets import mnist
from imutils import build_montages
import numpy as np
import cv2 as cv

In [2]:
def make_pairs(images, labels):
    # initialize 2 empty lists to hold (image, image) pairs and 
    # labels to indicate if a pair is positive or negative
    pair_images = []
    pair_labels = []
    
    # calculate the total number of classes present in dataset and then build a list of
    # indexes for each class label that provides indexes for all examples with a given label
    num_classes = len(np.unique(labels))
    idx = [np.where(labels==i)[0] for i in range(0, num_classes)]
    
    # loop over all images
    for idxA in range(len(images)):
        # grab the current image and label belonging to current iteration
        current_image = images[idxA]
        label = labels[idxA]
        
        # randomly pic an image that belong to same class label
        idxB = np.random.choice(idx[label])
        pos_image = images[idxB]
        
        # prepare positive pair and update images and labels lists respectively
        pair_images.append([current_image, pos_image])
        pair_labels.append([1])
        
        # grab the indicies for each of the class labels not equal to current label and 
        # randomly pick an image corresponding to a label not equal to current label
        neg_idx = np.where(labels!=label)[0]
        neg_image = images[np.random.choice(neg_idx)]
        
        # prepare a negative pair of images and update our lists
        pair_images.append([current_image, neg_image])
        pair_labels.append([0])
        
    return (np.array(pair_images), np.array(pair_labels))

In [3]:
# load the mnist dataset and scale pixel values to the range [0, 1]
print('[INFO] loading MNIST dataset...')
((trainX, trainY), (testX, testY)) = mnist.load_data()

# build the positive and negative image pairs
print('[INFO] preparing positive and negative pairs...')
(pair_train, label_train) = make_pairs(trainX, trainY)
(pair_test, label_test) = make_pairs(testX, testY)

# initialize the list of images that will be used when building our montages
images = []

# loop over a sample of training pairs
for i in np.random.choice(np.arange(0, len(pair_train)), size=(49,)):
    # grab the current image pair and label
    imageA = pair_train[i][0]
    imageB = pair_train[i][1]
    label = label_train[i]
    
    # to make it eaisier to visualize the pairs and their positive or negative annotations we are
    # going to pad the pairs with four pixels along the top, bottom and right boarders respectively
    output = np.zeros((36, 60), dtype='uint8')
    pair = np.hstack([imageA, imageB])
    output[4:32, 0:56] = pair
    
    # set the label for the pair along with the color we are going to draw the pair in (green for 
    # positive pair red for negative pair)
    text = 'pos' if label[0]==1 else 'neg'
    color = (0, 255, 0) if label[0]==1 else (0,0,255)
    
    # create a 3-channel RGB image from grascale pair, resize it from 28x28 to 96x51 and then draw
    # the type of pair it is on image
    vis = cv.merge([output]*3)
    vis = cv.resize(vis, (96, 51), interpolation=cv.INTER_LINEAR)
    cv.putText(vis, text, (2, 12), cv.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
    
    # add the pair visualization to our list of output images
    images.append(vis)
    
# construct the montage for images
montage = build_montages(images, (96, 51), (7, 7))[0]

# show the output montage
cv.imshow('Siamese Image Pairs', montage)
cv.waitKey(0)
cv.destroyAllWindows()

[INFO] loading MNIST dataset...
[INFO] preparing positive and negative pairs...
